# seq2seq

In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm

sys.path.append(os.path.abspath("../"))
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from transformers import get_constant_schedule

In [3]:
import importlib

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat
from seq2seq_modules.models import LSTMModel
from seq2seq_modules.weight_initialization import weights_init_uniform_rule
from seq2seq_modules.loops import cross_validation, single_model_training
from seq2seq_modules.utils import age_bucket
from seq2seq_modules.metrics import AGE_METRIC

## Read and process

In [4]:
SEQ2SEQ_DATA_PATH = '../seq2seq_data/'
DATA_PATH = '../data/'

SPLIT_SEED = 42

In [5]:
df = pandas_reduce_mem_usage(
    pd.read_parquet(f"{SEQ2SEQ_DATA_PATH}/stages/stage_6.parquet.gzip")
)
df.head()

Memory usage of dataframe is 28855.27 MB


  0%|          | 0/32 [00:00<?, ?it/s]

Memory usage after optimization is: 28855.27 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,geo_lat_y,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,timestamp,relative_timestamp
0,21,409,1,589,5788,2,1,20368.0,2,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,761,165528352.0,388.799988
1,21,409,1,589,12900,2,1,20368.0,2,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,549,165562912.0,43.200001
2,21,409,1,589,17626,2,1,20368.0,0,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165504608.0,626.239990
3,21,409,1,589,59366,2,1,20368.0,0,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165271328.0,2959.040039
4,21,409,1,589,59366,2,1,20368.0,0,1,...,45.040161,38.975964,1195.817871,1755.62085,3275.137451,1992.558472,6999.525391,712,165392288.0,1749.439941


In [6]:
target = pandas_reduce_mem_usage(
    pd.read_parquet(
        f'{DATA_PATH}public_train.pqt', columns=["user_id", "age"]
    )
)
target

Memory usage of dataframe is 6.18 MB


  0%|          | 0/2 [00:00<?, ?it/s]

Memory usage after optimization is: 4.12 MB
Decreased by 33.3%


,user_id,age
350459,350459,31.0
188276,188276,35.0
99002,99002,41.0
155506,155506,33.0
213873,213873,54.0
...,...,...
225374,225374,49.0
25776,25776,22.0
148131,148131,28.0
205570,205570,28.0


In [7]:
df = pandas_reduce_mem_usage(df.merge(target, how="left", on="user_id").dropna(subset=["age"]))
df.head()

Memory usage of dataframe is 19652.03 MB


  0%|          | 0/33 [00:00<?, ?it/s]

Memory usage after optimization is: 19652.03 MB
Decreased by 0.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,timestamp,relative_timestamp,age
2272,60,732,27,211,5790,2,0,74259.0,3,1,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,761,163262528.0,1058.239990,35.0
2273,60,732,27,211,65865,2,0,74259.0,0,3,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,549,162955808.0,4125.439941,35.0
2274,60,732,27,211,111474,2,0,74259.0,0,3,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,761,162947168.0,4211.839844,35.0
2275,14,311,27,211,111474,2,0,74259.0,1,2,...,20.507324,1092.055054,828.233154,3870.873535,2490.750244,7378.328613,761,163122112.0,2462.399902,35.0
2276,60,732,27,211,125409,2,0,74259.0,0,1,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,549,163033568.0,3347.840088,35.0


In [8]:
df["age"] = df["age"].progress_apply(age_bucket)
df = pandas_reduce_mem_usage(df)
df.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 214652540/214652540 [02:05<00:00, 1705562.37it/s]

Memory usage of dataframe is 20470.86 MB


  0%|          | 0/33 [00:00<?, ?it/s]

Memory usage after optimization is: 19037.90 MB
Decreased by 7.0%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,...,geo_lon_y,dist_to_Moscow,dist_to_SaintP,dist_to_Novosibirsk,dist_to_Ekaterinburg,dist_to_Vladivostok,domain,timestamp,relative_timestamp,age
2272,60,732,27,211,5790,2,0,74259.0,3,1,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,761,163262528.0,1058.239990,2
2273,60,732,27,211,65865,2,0,74259.0,0,3,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,549,162955808.0,4125.439941,2
2274,60,732,27,211,111474,2,0,74259.0,0,3,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,761,162947168.0,4211.839844,2
2275,14,311,27,211,111474,2,0,74259.0,1,2,...,20.507324,1092.055054,828.233154,3870.873535,2490.750244,7378.328613,761,163122112.0,2462.399902,2
2276,60,732,27,211,125409,2,0,74259.0,0,1,...,30.315901,635.758972,0.024454,3115.404541,1787.926147,6555.964355,549,163033568.0,3347.840088,2


In [10]:
df.to_parquet(f"{SEQ2SEQ_DATA_PATH}/version_4_age.parquet.gzip",
              compression='gzip')

## Make torch Dataset

In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm

sys.path.append(os.path.abspath("../"))
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import get_constant_schedule, get_cosine_schedule_with_warmup

In [3]:
import importlib

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat
from seq2seq_modules.models import LSTMModel, StarterBERTModel, AttentionPoolingBERTModel
from seq2seq_modules.data import TargetDataset
from seq2seq_modules.weight_initialization import weights_init_uniform_rule, weights_init_xavier
from seq2seq_modules.loops import cross_validation, single_model_training
from seq2seq_modules.utils import age_bucket
from seq2seq_modules.metrics import AGE_METRIC
from seq2seq_modules.utils import fix_random_state

In [4]:
LOCAL_DATA_PATH = '../seq2seq_data/'
SEED = 42
fix_random_state(SEED)

In [5]:
ids = ["user_id"]

cat_features = [
    "region_name",
    "city_name",
    "cpe_manufacturer_name",
    "cpe_model_name",
    "url_host",
    "cpe_type_cd",
    "cpe_model_os_type",
    "part_of_day",
    "domain",
    "capital_marker"
]

continous_features = [
    "request_cnt",
    "price",
    "timestamp",
    "relative_timestamp",
    "geo_lat_x",
    "geo_lon_x",
    "population",
    "timezone",
    "dist_to_Moscow",
    "dist_to_SaintP",
    "dist_to_Novosibirsk",
    "dist_to_Ekaterinburg",
    "dist_to_Vladivostok",
]

In [6]:
df = pandas_reduce_mem_usage(
    pd.read_parquet(f"{LOCAL_DATA_PATH}/version_4_age.parquet.gzip")
)

Memory usage of dataframe is 19037.90 MB


  0%|          | 0/33 [00:00<?, ?it/s]

Memory usage after optimization is: 19037.90 MB
Decreased by 0.0%


In [7]:
dataset = TargetDataset(
         df,
         agg_column="user_id", 
         time_column="timestamp",
         target_column="age",
         cat_features=cat_features,
         cont_features=continous_features,
         max_len=1024,
         padding_side="left",
)
dataset

  0%|          | 0/214652540 [00:00<?, ?it/s]

## Feed to the model

In [8]:
cat_feature_indexes = []
cont_feature_indexes = []
vocab_sizes = {}

for i in tqdm(range(len(cat_features))):
    cat_feature_indexes.append(i)
    vocab_sizes[i] = int(df[cat_features[i]].max() + 1)

for i in tqdm(range(len(continous_features))):
    cont_feature_indexes.append(i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 378652.44it/s]


In [9]:
vocab_sizes

{0: 81, 1: 985, 2: 37, 3: 599, 4: 199683, 5: 4, 6: 2, 7: 4, 8: 869, 9: 3}

In [10]:
targets = torch.cat([el[3].unsqueeze(0) for el in tqdm(dataset)], dim=0)
targets

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 269998/269998 [00:17<00:00, 15871.34it/s]


tensor([2, 3, 1,  ..., 2, 3, 3])

In [13]:
# model = AttentionPoolingBERTModel(
#         cat_feature_indexes=cat_feature_indexes,
#         vocab_sizes=vocab_sizes,
#         cont_feature_indexes=cont_feature_indexes,
#         encoder_hidden_dim=16,
#         hidden_dim=256,
#         dim_feedforward=512,
#         output_dim=7,
#         pe_type="trainable",
#         use_mask=False,
#         max_len=1024,
#         use_key_padding_mask=True,
# )

# model = StarterBERTModel(
#         cat_feature_indexes=cat_feature_indexes,
#         vocab_sizes=vocab_sizes,
#         cont_feature_indexes=cont_feature_indexes,
#         encoder_hidden_dim=16,
#         hidden_dim=256,
#         dim_feedforward=512,
#         output_dim=7,
#         pe_type="trainable",
#         use_mask=False,
#         max_len=1024,
#         use_key_padding_mask=False,
#         starter="randn",
#         shared=False
# )

model = LSTMModel(
        cat_feature_indexes=cat_feature_indexes,
        vocab_sizes=vocab_sizes,
        cont_feature_indexes=cont_feature_indexes,
        encoder_hidden_dim=16,
        hidden_dim=256,
        output_dim=7,
)


weights_init_xavier(model)

loss = nn.CrossEntropyLoss(
    weight=torch.tensor(
        compute_class_weight(
            class_weight="balanced",
            classes=sorted(df["age"].unique()),
            y=targets.numpy()
        )
    )
)

metric = AGE_METRIC

In [14]:
fold_train_scores, fold_eval_scores = cross_validation(
        project_name="age_LSTMModel_v1", 
        model=model, 
        dataset=dataset, 
        loss_function=loss, 
        metric_func=AGE_METRIC, 
        optimizer=torch.optim.AdamW, 
        get_scheduler=get_cosine_schedule_with_warmup, 
        strat_array=targets.numpy(), 
        device='cuda', 
        random_state=69, 
        shuffle=True, 
        dataloader_shuffle=False, 
        n_folds=5, 
        epochs=10, 
        lr=1e-3, 
        weight_decay=1e-4,
        start_fold=0, 
        batch_size=32,
)

FOLD 0
--------------------------------


  0%|          | 0/6750 [00:00<?, ?it/s]

KeyboardInterrupt: 